In [5]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立Ollama模型
model = OllamaLLM(model="llama3.2:latest")
print("模型已建立")

模型已建立


In [10]:
# 第一步:生成郵件回覆
reply_prompt = ChatPromptTemplate.from_messages([
    ("system","你是專業的客服代表。請針對客戶的問題提供簡潔、專業的回覆內容。"),
    ("human","客戶郵件：\n\n{email_content}")
])

# 第二步:格式化郵件
format_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "請將以下郵件回覆格式化，加上適當的稱呼、問候語和專業的結尾。"),
        ("human", "郵件內容：\n\n{reply}")
    ]
)

# 第三步: 品質檢查和改進建議
improve_prompt = ChatPromptTemplate.from_messages([
    ("system","是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
    ("human", "評估並改進這封郵件：\n\n{formatted_reply}")
])

print("三階段 prompt已定義完成")

三階段 prompt已定義完成


In [11]:
sequential_chain = (
    reply_prompt
    | model
    | StrOutputParser()
    | (lambda x:{"reply":x})
    | format_prompt
    | model
    | StrOutputParser()
    | (lambda x: {"formatted_reply":x})
    | improve_prompt
    | model
    | StrOutputParser()
)
print("✅ 串聯模型鏈已建立完成！")
print("\n流程：生成回覆 → 格式化郵件 → 品質評估與改進建議")

✅ 串聯模型鏈已建立完成！

流程：生成回覆 → 格式化郵件 → 品質評估與改進建議


In [13]:
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

result = sequential_chain.invoke({"email_content":customer_email})
print(result)

📨 專業度：8/10

* 導入了適當的 Formality 和 Politeness Language（例如“尊敬的王先生”、“ cordially greetings”）
* 提到了退款程序和換貨流程，但缺乏詳細信息
* 使用簡潔的語句和步驟，易於閱讀

友善度：9/10

* 導入了-friendly Language（例如“祝福您一天！”）
* 提到了幫助和聯繫方式，是友善的

完整度：7/10

* 缺乏明確的連結或資源，讓收件者需要進一步查找更多信息
* some 步驟缺乏詳細描述，例如 Courier Courier 回送服務的費用和流程
* 導入了多餘的字句（例如“請 cordially greetings”）

改進建議：

1. 增加清晰度：提供明確的連結或資源，以幫助收件者查找更多信息。
2. 加強詳細性：在每個步驟中提供更多的描述，例如 Courier Courier 回送服務的費用和流程。
3. 刪除多餘字句：減少使用複雜的語句或表達式。

改過後的封信：

尊敬的王先生，

感謝您對我們公司的連絡。因為包裝損壞，您可以按照以下步驟進行：

1. 請您在寄出包裹後3天內寄出原來包裹用的商品回送，請附上您購買項目的購物車號、收款信息及您當前的收件地址。
2. 我們會根據原價支付退款。如果您的貨物已經配送過，那麼我們也會提供 Courier Courier 回送服務，費用由我們負責。詳情請參考以下連結：[連結]
3. 如需要換貨，您也可以要求我們進行換貨。

如果您有任何疑問或需要進一步幫助，请不誤與我們聯繪。

祝福您一天！

(Note：以下改變僅供示範，實際的封信應根據公司的 branding 和風格進行調整)
